In [94]:
# Imports
import csv
import os

FILE_PATH = 'C:\\Users\\jona1\\github\\MerismaCapture\\src\\ss_module\\dataset\\'
CIK_IDENTIFIERS = [
    '0001720792',
    '0001099281',
    '0001079114',
    '0001112520',
    '0001641864',
    '0000846222',
    '0001709323',
    '0000732905',
    '0000883965',
    '0001067983',
    '0001061768',
]

From the SUBMISSION table fetch a list of ACCESSION_NUMBER(s) using the CIK identifiers in table A-1 (Appendix).




In [95]:

picked_submissions = []
with open(FILE_PATH + 'SUBMISSION_2023_q1.tsv', 'r', encoding='utf-8') as q1:
    for submission in csv.DictReader(q1, delimiter="\t"):
        if submission["CIK"] in CIK_IDENTIFIERS:
            picked_submissions.append(submission["ACCESSION_NUMBER"])

with open(FILE_PATH + 'SUBMISSION_2023_q2.tsv', 'r', encoding='utf-8') as q2:
    for submission in csv.DictReader(q2, delimiter="\t"):
        if submission["CIK"] in CIK_IDENTIFIERS:
            picked_submissions.append(submission["ACCESSION_NUMBER"])

with open(FILE_PATH + 'SUBMISSION_2023_q3.tsv', 'r', encoding='utf-8') as q3:
    for submission in csv.DictReader(q3, delimiter="\t"):
        if submission["CIK"] in CIK_IDENTIFIERS:
            picked_submissions.append(submission["ACCESSION_NUMBER"])
    

print(len(picked_submissions))


35


From the INFOTABLE fetch a list of NAMEOFISSUER(s) using the ACCESSION_NUMBER(s) created in (b). Use CUSIP(s) to map between both.